<a href="https://colab.research.google.com/github/nickhitt/AMEX-fraud/blob/master/Amex_Credit_Default.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## American Express Fraud Detection Competition

Credit default prediction is central to managing risk in a consumer lending business. Credit default prediction allows lenders to optimize lending decisions, which leads to a better customer experience and sound business economics.

In this competition, I apply my machine learning skills to predict credit default for American Express. Specifically, I leverage an industrial scale data set to build a machine learning model that challenges the current model in production. Training, validation, and testing datasets include time-series behavioral data and anonymized customer profile information. 

This notebook is organised according to the following sections.

1. Loading the data
2. Exploratory Data Analysis 
3. Pre-Processing/Feature Engineering
4. Train-Test Split
5. Model Selection
6. Model Tuning
7. Predictive Output



## 1.0 Data Loading

First lets download the data into Colab from Kaggle

In [3]:
from google.colab import files
uploaded = files.upload()

Saving kaggle.json to kaggle.json


In [4]:
!pwd

/content


In [7]:
import os
os.environ['KAGGLE_CONFIG_DIR'] = "/content"

In [8]:
!kaggle competitions download -c amex-default-prediction


100% 20.4G/20.5G [01:55<00:00, 60.2MB/s]
100% 20.5G/20.5G [01:55<00:00, 190MB/s] 


Next let's unzip the file

In [ ]:
!unzip /content/amex-default-prediction.zip

Archive:  /content/amex-default-prediction.zip
  inflating: sample_submission.csv   
  inflating: test_data.csv           

Now let's read the data into our workspace

In [ ]:
import pandas as pd
train = pd.read_csv("/content/test_data.csv")
test = pd.read_csv("/content/test_data.csv")
train_labels = pd.read_csv("/content/train_labels.csv")

## 2.0 Exploratory Data Analysis
Let's have a look around the data and take a look at what we're dealing with

In [ ]:
train.info()

In [ ]:
train.head()

In [ ]:
train['customer_ID'].nunique()